In [ ]:
from multiprocessing import cpu_count, Pool
import os
from pathlib import Path
import shutil

from killscreen.monitors import Netstat, Stopwatch
from pyarrow import parquet

# hacky; can remove if we decide to add an install script or put this in the repo root
os.chdir(globals()['_dh'][0].parent)

from subset.science.handlers import (
    filter_ps1_catalog, sample_ps1_catalog, get_corresponding_images,
    bulk_skycut
)
from subset.science.ps1_utils import ps1_stack_path, request_ps1_cutout, \
    PS1_CUT_CONSTANTS
from subset.utilz.generic import make_loaders, parse_topline
from subset.utilz.mount_s3 import mount_bucket

In [ ]:
# what bucket are our images and metadata files stored in?
BUCKET = 'nishapur'
# where, on the local filesystem, shall we create a FUSE mount for that bucket?
S3_ROOT = '/mnt/s3'
# mount that bucket to read metadata
mount_bucket(remount=False, mount_path=S3_ROOT, bucket=BUCKET)
# catalog of all mean objects from 1000 PS1 sky cells randomly selected from
# "extragalactic" cells that overlap the viewports of GALEX visits, then filtered
# to the "best" objects (qualityFlag bit 0b100000) with valid photometry in both
# g and z bands (this filter leaves roughly 3% of total sources). other
# similarly-formatted catalog files can be used.
CATALOG_FN = "ps1_eg_eclipses_subset_best_gz_coregistered.parquet"
if not Path(CATALOG_FN).exists():
    shutil.copy(Path(S3_ROOT, "ps1/metadata", CATALOG_FN), Path(CATALOG_FN))
catalog = parquet.read_table(CATALOG_FN).to_pandas()
# cutouts in dimensions: ra, dec in degrees. treated as side lengths of a rectangle.
CUT_SHAPE = (60 / 3600, 60 / 3600)
# restrict to sources bright in both g and z? set to 'None' for no cutoff.
MAG_CUTOFF = 20
# restrict to only sources flagged as extended / not extended?
# "extended", "point", or None for no restriction
EXTENSION_TYPE = None
# restrict to only sources with a valid stack detection? (probably a good idea)
STACK_ONLY = True
# which PS1 bands are we considering? (only g and z are currently staged, but you can stage more.)
PS1_BANDS = ("g", "z")
# how many targets shall we randomly select?
TARGET_COUNT = 30
# optional parameter -- restrict the total number of PS1 source cells to test the
# performance effects of denser sampling. 1000 total cells are available in this test set.
# note that the total number of images accessed is number of cells * number of bands.
MAX_CELL_COUNT = 8
# select loaders -- options are "astropy", "fitsio", "greedy_astropy", "greedy_fitsio"
# NOTE: because all the files this particular notebook is looking
# at are RICE-compressed, there is unlikely to be much difference
# between astropy and greedy_astropy -- astropy does not support
# loading individual tiles from a a tile-compressed FITS file.
LOADERS = make_loaders("fitsio",)

In [ ]:
# all sources that fit characteristic criteria
candidate_sources = filter_ps1_catalog(catalog, MAG_CUTOFF, EXTENSION_TYPE, STACK_ONLY)
# randomly-selected subset of those sources w/adequate metadata for cutout definition
targets = sample_ps1_catalog(candidate_sources, TARGET_COUNT, MAX_CELL_COUNT)
# add requested cut shape instructions to these target definitions
targets = [t | {'ra_x': CUT_SHAPE[0], 'dec_x': CUT_SHAPE[1]} for t in targets]
# make lists of the ps1 stack images these sources lie within
# (so that we can easily initialize each relevant image only once)
ps1_stacks, _ = get_corresponding_images(targets)

In [ ]:
# per-loader performance-tuning parameters.
# image_chunksize: how many images shall we initialize at once?
# image_threads: how many threads shall we init with in parallel? (None to disable.)
# cut_threads: how many threads shall we cut with in parallel? (None to disable.)
# note that S3 handles parallel requests very well; on a smaller instance, you will
# usually run out of CPU or absolute input bandwidth before you exhaust its willingness to
# serve parallel requests.
TUNING = {
    "fitsio": {
        "image_chunksize": 40, "image_threads": cpu_count() * 7, "cut_threads": cpu_count() * 7
    },
    "greedy_fitsio": {
        "image_chunksize": 10, "image_threads": cpu_count() * 2, "cut_threads": None
    },
    "default": {
        "image_chunksize": 20, "image_threads": cpu_count() * 4, "cut_threads": cpu_count() * 4
    },
}

In [ ]:
logs = {}
for loader_name, loader in LOADERS.items():
    # remount bucket to avoid "cheating" -- note that this is still a little cheaty
    # because of unreliable, unmodifiable, and entirely black-box caching on S3 side, and loaders
    # later in the list will tend to do better. for a 'fairer' comparison, reroll
    # between each loader or juice serverside caching with throwaway tests (see benchmarking suite)
    print(f"----testing {loader_name}----")
    mount_bucket(remount=True, mount_path=S3_ROOT, bucket=BUCKET)
    tuning_params = TUNING[loader_name] if loader_name in TUNING.keys() else TUNING["default"]
    cuts, logs[loader_name] = bulk_skycut(
        ps1_stacks,
        targets,
        loader=loader,
        return_cuts=True,
        data_root=f"{S3_ROOT}/ps1",
        bands=PS1_BANDS,
        verbose=2,
        **PS1_CUT_CONSTANTS,
        **tuning_params
    )
    rate, weight = parse_topline(logs[loader_name])
    print(f"{rate} cutouts/s, {weight} MB / cutout")

In [ ]:
# comparison to the PS1 cutout service. We can crank up the number of threads we're using...
# but at some point we will essentially be attacking the service; no one else will be able to use it
# (or it will be forced to block us).
# also note that the PS1 cutout service _also_ performs serverside caching, so executing
# multiple requests in a row against the same group of images will result in increased performance.

REQUEST_THREADS = None
watch, netstat = Stopwatch(silent=True), Netstat()
watch.start(), netstat.update()
req_cutouts = {}
request_pool = Pool(REQUEST_THREADS) if REQUEST_THREADS is not None else None

for target in targets:
    for band in PS1_BANDS:
        args = (
            ps1_stack_path(target['proj_cell'], target['sky_cell'], band),
            target['ra'],
            target['dec'],
            (CUT_SHAPE[0] + CUT_SHAPE[1]) / 2 * 3600,
            "fits"
        )
        if request_pool is None:
            req_cutouts[target['obj_id']] = request_ps1_cutout(*args)
        else:
            req_cutouts[target['obj_id']] = request_pool.apply_async(
                request_ps1_cutout, args
            )
if request_pool is not None:
    req_cutouts = {
        obj_id: result.get() for obj_id, result in req_cutouts.items()
    }
netstat.update()
count = len(targets) * len(PS1_BANDS)
sec = watch.peek()
vol = list(netstat.total.values())[-1] / 1024 ** 2
print(
    f"made {count} cutouts,{sec} total seconds,{round(vol, 2)} total MB,\n"
    f"{round(count / sec, 2)} cutouts / s,{round(vol / count, 2)} MB/cutout"
)